Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс.)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

!!!В первую очередь делаем сайт hh.ru - его обязательно. sj.ru можно попробовать сделать вторым. Он находится в очень странном состоянии и возможна некорректная работа.!!!

In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import re

In [2]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'}

In [3]:
def hh(search_str, n_str):
    main_link = 'https://hh.ru/'
    html = requests.get(main_link + '/search/vacancy?area=1&st=searchVacancy&text=' + search_str + '&fromSearch=true&from=suggest_post', headers = headers).text
    parsed_html = bs(html,'lxml')

    jobs = []
    for i in range(n_str):
        jobs_block = parsed_html.find('div', {'class' : 'vacancy-serp-item__row vacancy-serp-item__row_header'})
        jobs_list = jobs_block.findChildren(recursive = False)
        for job in jobs_list:
            job_data = {}
            req = job.find('span', {'class' : 'g-user-content'})
            if req != None:
                main_info = req.findChild()
                job_name = main_info.getText()
                job_link = main_info['href']
                salary = job.find('span', {'class': 'vacancy-serp-item__compensation'})
                
                if not salary:
                    salary_min = None
                    salary_max = None
                    salary_currency = None
                else:
                    salary = salary.getText().replace(u'\xa0', u'')

                    salary = re.split(r'\s|-', salary)

                    if salary[0] == 'до':
                        salary_min = None
                        salary_max = int(salary[1])
                    elif salary[0] == 'от':
                        salary_min = int(salary[1])
                        salary_max = None
                    else:
                        salary_min = int(salary[0])
                        salary_max = int(salary[1])            

                    salary_currency = salary[2]
                    
                job_data['name'] = job_name
                job_data['salary_min'] = salary_min
                job_data['salary_max'] = salary_max
                job_data['salary_currency'] = salary_currency
                job_data['link'] = job_link
                job_data['site'] = main_link
                jobs.append(job_data)
                
        next_btn_block = parsed_html.find('a', {'class' : 'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})
        next_btn_link = next_btn_block['href']
        html = requests.get(main_link + next_btn_link, headers = headers).text
        parsed_html = bs(html, 'lxml')

    pprint(jobs)
    return jobs

In [4]:
search_str = 'Data science'
n_str = 3

hh(search_str, n_str)

[{'link': 'https://hh.ru/vacancy/36916080?query=Data%20science',
  'name': 'Python Data Scientist',
  'salary_currency': None,
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/'},
 {'link': 'https://hh.ru/vacancy/37393665?query=Data%20science',
  'name': 'Release Manager',
  'salary_currency': None,
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/'},
 {'link': 'https://hh.ru/vacancy/37332883?query=Data%20science',
  'name': 'Преподаватель курса "Анализ данных и машинное обучение"',
  'salary_currency': None,
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/'}]


[{'name': 'Python Data Scientist',
  'salary_min': None,
  'salary_max': None,
  'salary_currency': None,
  'link': 'https://hh.ru/vacancy/36916080?query=Data%20science',
  'site': 'https://hh.ru/'},
 {'name': 'Release Manager',
  'salary_min': None,
  'salary_max': None,
  'salary_currency': None,
  'link': 'https://hh.ru/vacancy/37393665?query=Data%20science',
  'site': 'https://hh.ru/'},
 {'name': 'Преподаватель курса "Анализ данных и машинное обучение"',
  'salary_min': None,
  'salary_max': None,
  'salary_currency': None,
  'link': 'https://hh.ru/vacancy/37332883?query=Data%20science',
  'site': 'https://hh.ru/'}]